# 03: Data cleaning and merge

This notebook combines StatCan and CMHC outputs into a single modeling table. It also checks how well metro names line up across sources so we can see where matches fail.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path


## Metro master crosswalk

The master crosswalk gives each metro a stable `metro_id` and a clean name. This lets different datasets join reliably even when labels differ slightly.

In [2]:
# Load processed metro tables + missingness diagnostics
from pathlib import Path

PROJECT = Path.cwd().resolve()
if PROJECT.name == "notebooks":
    PROJECT = PROJECT.parent
PROCESSED = PROJECT / "data" / "processed"

metros_master = pd.read_csv(PROCESSED / "metros_master.csv")
missing_summary = pd.read_csv(PROCESSED / "metro_join_missingness_summary.csv")
missing_unmatched = pd.read_csv(PROCESSED / "metro_join_missingness_unmatched.csv")

print(f"metros_master shape: {metros_master.shape}")
metros_master.head()

missing_summary

missing_unmatched

metros_master shape: (41, 7)


,geo_label,metro_name_source,province_source,metro_slug,is_partial,metro_id,matched,source
0,"Ottawa-Gatineau, Ontario part, Ontario/Quebec",Ottawa-Gatineau,Ontario/Quebec,ottawa_gatineau,True,NaN,False,statcan_unemployment
1,"Ottawa-Gatineau, Quebec part, Ontario/Quebec",Ottawa-Gatineau,Ontario/Quebec,ottawa_gatineau,True,NaN,False,statcan_unemployment


## Modeling dataset (StatCan + CMHC)

We keep only metros that appear in both sources (inner join). Missing values are not filled in yet; they are left as blanks so you can see where data coverage is thin.

In [3]:
# Build modeling dataset from StatCan + CMHC outputs
statcan = pd.read_csv(PROCESSED / "statcan_metro.csv")
cmhc = pd.read_csv(PROCESSED / "cmhc_metro.csv")

def normalize_metro_id(series):
    return (
        series.astype(str)
        .str.strip()
        .str.replace(r"\.0$", "", regex=True)
        .str.zfill(4)
    )

statcan["metro_id"] = normalize_metro_id(statcan["metro_id"])
cmhc["metro_id"] = normalize_metro_id(cmhc["metro_id"])
metros_master["metro_id"] = normalize_metro_id(metros_master["metro_id"])

base = metros_master[["metro_id", "metro_name_std", "province", "metro_slug", "province_slug"]]
statcan_features = statcan.drop(columns=["metro_name_std", "province"])
cmhc_features = cmhc.drop(columns=["metro_name_std", "province"])

modeling = (
    base.merge(statcan_features, on="metro_id", how="left")
        .merge(cmhc_features, on="metro_id", how="left")
)

# Keep metros present in both sources; allow partial feature missingness.
modeling = modeling[
    modeling["statcan_reference_year"].notna() & modeling["cmhc_rent_year"].notna()
].copy()

modeling.sort_values("metro_name_std", inplace=True)

feature_cols = [
    c for c in modeling.columns
    if c not in {"metro_id", "metro_name_std", "province", "metro_slug", "province_slug"}
]

retained_summary = pd.DataFrame(
    {"metric": ["metros_total", "metros_retained"],
     "value": [len(metros_master), len(modeling)]}
)

missing_by_feature = pd.DataFrame(
    {"feature": feature_cols,
     "missing_count": modeling[feature_cols].isna().sum().values}
)
missing_by_feature["missing_pct"] = (
    missing_by_feature["missing_count"] / len(modeling)
).round(3)

retained_summary
missing_by_feature.sort_values("missing_count", ascending=False)

output_path = PROCESSED / "metros_modeling.csv"
modeling.to_csv(output_path, index=False)
output_path


PosixPath('/Users/andrewharris/Projects/housing-affordability/data/processed/metros_modeling.csv')